# Step 1: finding candidates

### Our client is a tech start-up that wants to be near similar companies, 
### so we look for web-sector companies in the Crunchbase(R) database:

In [ ]:
from pymongo import MongoClient

In [1]:
import pandas as pd
import numpy as np

In [ ]:
from src import api_functions as af

In [ ]:
client = MongoClient("localhost:27017")

In [ ]:
db = client.get_database("IronHack")

In [ ]:
c = db.get_collection("companies")

In [ ]:
proj = {"_id":0, "name":1, "category_code":1, "offices":1}

In [ ]:
compa = list(c.find({"category_code": "web"},proj))

In [ ]:
df=pd.DataFrame(compa)

In [ ]:
df.shape

In [ ]:
df["city"]=df.apply(lambda fila : fila.offices[0]["city"] if len(fila.offices)>0 else "nulo" , axis=1)

In [ ]:
df["zip_code"]=df.apply(lambda fila : fila.offices[0]["zip_code"] if len(fila.offices)>0 else "nulo" , axis=1)

### and we get the 10 most repeated zip codes within those companies:

In [ ]:
top = df.zip_code.value_counts().head(12)

(we skip the "nulo" and blank spaces)

In [ ]:
zips=list(top.index[2:])

In [ ]:
zips

### Then, we get the coords for those zip codes from the US postal service API:

(we use the "get coord from zip" function from our API_functions file)

In [ ]:
coords = [af.get_coord_from_zip(elem)  for elem in zips]

In [ ]:
df = pd.DataFrame(coords)
df.drop("state_fullname", axis=1,inplace=True)
df

### and we export this data to a csv:

In [ ]:
df.to_csv("DATA/candidates.csv")

## Bonus: showing candidates in a map:

In [5]:
df = pd.read_csv("DATA/candidates.csv")          # por si el limite diario de la API da problemas
df.drop("Unnamed: 0", axis=1,inplace=True)

In [2]:
import folium

In [11]:
mymap = folium.Map(location=[38.75,-98.79], zoom_start=4, tiles="cartodbpositron")

In [12]:
for i in range(10):
    mark = folium.Marker(location=[df.iloc[i].latitude, df.iloc[i].longitude], tooltip = f"{zips[i]}")
    mark.add_to(mymap)

In [13]:
mymap

### y lo guardamos:

In [16]:
mymap.save("DATA/candidates_map")